**Students Database Mongodb task**

In [21]:
import pymongo
import json
client=pymongo.MongoClient("mongodb://127.0.0.1:27017")

In [5]:
db=client["Student_database"]     ## creating a database     

In [6]:
stdnts=db['students']          ## creating a collection

In [7]:
with open('D:\\data\\test_3.json','r') as file:
    file_data = json.load(file)

In [27]:
stdnts.insert_many(file_data)

In [ ]:
my_dbs=client.list_database_names() ## to read existing databases in mongodb
print(my_dbs)

In [ ]:
#####################################################for getting average of exam scores

In [75]:
agg_result= stdnts.aggregate([
   {
     "$unwind": "$scores"
   },
   {
      "$match": { "scores.type": "exam" }
   },
   {
      "$group": {
         "_id": "null",
         "averageScore": { "$avg": "$scores.score" }
      }
   }
])
for i in agg_result:
    avg_exam=float(i.get("averageScore"))
print(avg_exam)


48.67367075950175


In [ ]:
######################################  find max score of exam 

In [41]:
max_result=stdnts.aggregate([
   {
     "$unwind": "$scores"
   },
   {
      "$match": { "scores.type": "exam" }
   },
   {
      "$group": {
         "_id": "null",
         "maxScore": { "$max": "$scores.score" }
      }
   }
])
for i in max_result:
     max_exam=float(i.get("maxScore"))

99.33685767140612


In [ ]:
####### max for quiz

In [38]:
max_result1=stdnts.aggregate([
   {
     "$unwind": "$scores"
   },
   {
      "$match": { "scores.type": "quiz" }
   },
   {
      "$group": {
         "_id": "quiz",
         "maxScore": { "$max": "$scores.score" }
      }
   }
])
for i in max_result1:
    max_quiz=float(i.get("maxScore"))

99.80348240553108


In [ ]:
#### max for homework

In [74]:
max_result2=stdnts.aggregate([
   {
     "$unwind": "$scores"
   },
   {
      "$match": { "scores.type": "homework" }
   },
   {
      "$group": {
         "_id": "homework",
         "maxScore": { "$max": "$scores.score" }
      }
   }
])
for i in max_result2:
     max_homework=float(i.get("maxScore"))
print(max_homework)

99.77237745070993


In [ ]:
##############################################3

**1)Find the student name who scored maximum scores in all (exam, quiz and homework)?**

In [108]:
result=stdnts.aggregate([
   {
     "$unwind": "$scores"
   },
   {
      "$match": {
         "scores.type": "exam",
         "scores.score": { "$gte": max_exam }
      }
   },
   {
      "$match": {
         "scores.type": "quiz",
         "scores.score": { "$gte": max_quiz }
      }
   },
   {
      "$match": {
         "scores.type": "homework",
         "scores.score": { "$gte": max_homework }
      }
   },
   {
      "$group": {
         "_id": "$name",
         "count": { "$sum": 1 }
      }
   },
   {
      "$match": {
         "count": 3
      }
   }
])
for i in result:
    print(i)

**2)      Find students who scored below average in the exam and pass mark is 40%?**

In [ ]:
x=stdnts.aggregate([
   {
      "$unwind": "$scores"
   },
   {
      "$match": {
         "scores.type": "exam"
      }
   },
   {
      "$group": {
         "_id": "$name",
         "exam_score": { "$sum": "$scores.score" }
      }
   },
   {
      "$match": {
         "exam_score": { "$lt": avg_exam}
      }
   }
])
for i in x:
    print(i)

**3)      Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.**

In [ ]:
THRESHOLD = 40
# Update all documents to set the "status" field to "pass"
result = stdnts.update_many({}, {"$set": {"status": "pass"}})

# Print the number of documents updated
print(result.modified_count)

# Update the documents and set the "status" field to "fail" if the score is below the threshold
result = stdnts.update_many(
    {"scores": {"$elemMatch": {"type": "exam", "score": {"$lt":THRESHOLD}}}},
    {"$set": {"status": "fail"}}
)
print(f"{result.modified_count} documents updated for exams")

result = stdnts.update_many(
    {"scores": {"$elemMatch": {"type": "quiz", "score": {"$lt":THRESHOLD}}}},
    {"$set": {"status": "fail"}}
)
print(f"{result.modified_count} documents updated for quizzes")

result = stdnts.update_many(
    {"scores": {"$elemMatch": {"type": "homework", "score": {"$lt":THRESHOLD}}}},
    {"$set": {"status": "fail"}}
)
print(f"{result.modified_count} documents updated for homework")

**4)       Find the total and average of the exam, quiz and homework and store them in a separate collection**

In [ ]:
##Didnt understand the above question 
#will solve once question is explained

**5)      Create a new collection which consists of students who scored below average and above 40% in all the categories.**

In [ ]:
# Calculate the average scores for each type of score
pipeline = [
    {"$unwind": "$scores"},
    {"$group": {"_id": "$scores.type", "avg_score": {"$avg": "$scores.score"}}},
]
avg_scores = list(stdnts.aggregate(pipeline))

# Create a dictionary with the average scores
avg_score_dict = {doc["_id"]: doc["avg_score"] for doc in avg_scores}
print(avg_score_dict)
# Filter the documents based on the average scores
query = {
    "scores": {
        "$elemMatch": {
            "$and": [
                {"type": {"$in": ["exam", "quiz", "homework"]}},
                {"score": {"$gt": 40}},
                {"score": {"$lt": avg_score_dict["type"]}},
            ]
        }
    }
}
filtered_docs = stdnts.find(query)

# Create a new collection with the filtered documents
below_avg_pass_col = db["below_avg_pass"]
below_avg_pass_col.insert_many(filtered_docs)

**6)Create a new collection which consists of students who scored below the fail mark in all the categories.**

In [ ]:
# Create the new collection
failed_collection = db["failed_students"]

# Iterate through the documents in the collection
for doc in stdnts.find({}):
    # Check if the student failed the exam, quiz, and homework
    failed = True
    for score in doc["scores"]:
        if score["type"] == "exam" and score["score"] >= 40:
            failed = False
            break
        elif score["type"] == "quiz" and score["score"] >= 40:
            failed = False
            break
        elif score["type"] == "homework" and score["score"] >= 40:
            failed = False
            break
    # If the student failed all exams, insert the document into the new collection
    if failed:
        failed_collection.insert_one(doc)

# Print the number of documents in the new collection
print(failed_collection.count_documents({}))
for document in failed_collection.find():
    print(document)

**7)      Create a new collection which consists of students who scored above pass mark in all the categories.**

In [ ]:
# Create the new collection
passed_collection = db["passed_students"]

# Iterate through the documents in the collection
for doc in stdnts.find({}):
    # Check if the student passed the exam, quiz, and homework
    passed = True
    for score in doc["scores"]:
        if score["type"] == "exam" and score["score"] < 40:
            passed = False
            break
        elif score["type"] == "quiz" and score["score"] < 40:
            passed = False
            break
        elif score["type"] == "homework" and score["score"] < 40:
            passed = False
            break
    # If the student failed all exams, insert the document into the new collection
    if passed:
        passed_collection.insert_one(doc)

# Print the number of documents in the new collection
print(passed_collection.count_documents({}))
for document in passed_collection.find():
    print(document)

**Telephone Directory CRUD Operation mongodb task**

In [1]:
import pymongo
client=pymongo.MongoClient("mongodb://127.0.0.1:27017")

In [2]:
db=client["Telephone_directory"]     ## creating a database     

In [3]:
contact_details=db['Contact_Details']          ## creating a collection

In [13]:
contact_info=[{'name': 'Arjun','phone_number':9845755542,'city':'Nanded'},
              {'name': 'Dittya Thakre','phone_number':7854857542,'city':'Pune'},
              {'name': 'Ranjeet Chaudhry','phone_number':8457961234,'city':'Mumbai'},
              {'name': 'Suman Pandey','phone_number':9845875462,'city':'Ahmednagar'},
              {'name': 'Rishabh Pant','phone_number':8754693214,'city':'Solapur'},
              {'name': 'Taehyung Kim','phone_number':5697541247,'city':'Daegu'},
              {'name': 'Park Jimin','phone_number':6589741237,'city':'Busan'},
              {'name': 'Jungkook John','phone_number':8547962541,'city':'Busan'},
              {'name': 'Seokjin Kim','phone_number':4568721397,'city':'Seoul'},
              {'name': 'Namjoon Kim','phone_number':2564781399,'city':'Seoul'}
              ]
contact_information=contact_details.insert_many(contact_info)  ## inserting many docs at a time
print(contact_information)
# for document in contact_details:
#    print(document)

In [14]:
my_dbs=client.list_database_names() ## to read existing databases in mongodb
print(my_dbs)

['GUVI1', 'IIT', 'Student_database', 'Telephone_directory', 'admin', 'config', 'guvi', 'local']


In [15]:
for document in contact_details.find({},{'_id':0}):  ##query to find records 
    print(document) 

{'name': 'Arjun', 'phone_number': 9845755542, 'city': 'Nanded'}
{'name': 'Dittya Thakre', 'phone_number': 7854857542, 'city': 'Pune'}
{'name': 'Ranjeet Chaudhry', 'phone_number': 8457961234, 'city': 'Mumbai'}
{'name': 'Suman Pandey', 'phone_number': 9845875462, 'city': 'Ahmednagar'}
{'name': 'Rishabh Pant', 'phone_number': 8754693214, 'city': 'Solapur'}
{'name': 'Taehyung Kim', 'phone_number': 5697541247, 'city': 'Daegu'}
{'name': 'Park Jimin', 'phone_number': 6589741237, 'city': 'Busan'}
{'name': 'Jungkook John', 'phone_number': 8547962541, 'city': 'Busan'}
{'name': 'Seokjin Kim', 'phone_number': 4568721397, 'city': 'Seoul'}
{'name': 'Namjoon Kim', 'phone_number': 2564781399, 'city': 'Seoul'}


In [16]:
for document in contact_details.find({'name':{'$gt':'G'}},{'_id':0}):  ## query to find records by giving conditions
    print(document)  

{'name': 'Ranjeet Chaudhry', 'phone_number': 8457961234, 'city': 'Mumbai'}
{'name': 'Suman Pandey', 'phone_number': 9845875462, 'city': 'Ahmednagar'}
{'name': 'Rishabh Pant', 'phone_number': 8754693214, 'city': 'Solapur'}
{'name': 'Taehyung Kim', 'phone_number': 5697541247, 'city': 'Daegu'}
{'name': 'Park Jimin', 'phone_number': 6589741237, 'city': 'Busan'}
{'name': 'Jungkook John', 'phone_number': 8547962541, 'city': 'Busan'}
{'name': 'Seokjin Kim', 'phone_number': 4568721397, 'city': 'Seoul'}
{'name': 'Namjoon Kim', 'phone_number': 2564781399, 'city': 'Seoul'}


In [17]:
old_data={'name': 'Arjun'}                         ## updating the existing data using update_one query
new_data={"$set":{'name': 'Arjun Pandey'}}
contact_details.update_one(old_data,new_data)
for document in contact_details.find():
    print(document)

{'_id': ObjectId('63b59a698291698b1cd1a77a'), 'name': 'Arjun Pandey', 'phone_number': 9845755542, 'city': 'Nanded'}
{'_id': ObjectId('63b59a698291698b1cd1a77b'), 'name': 'Dittya Thakre', 'phone_number': 7854857542, 'city': 'Pune'}
{'_id': ObjectId('63b59a698291698b1cd1a77c'), 'name': 'Ranjeet Chaudhry', 'phone_number': 8457961234, 'city': 'Mumbai'}
{'_id': ObjectId('63b59a698291698b1cd1a77d'), 'name': 'Suman Pandey', 'phone_number': 9845875462, 'city': 'Ahmednagar'}
{'_id': ObjectId('63b59a698291698b1cd1a77e'), 'name': 'Rishabh Pant', 'phone_number': 8754693214, 'city': 'Solapur'}
{'_id': ObjectId('63b59a698291698b1cd1a77f'), 'name': 'Taehyung Kim', 'phone_number': 5697541247, 'city': 'Daegu'}
{'_id': ObjectId('63b59a698291698b1cd1a780'), 'name': 'Park Jimin', 'phone_number': 6589741237, 'city': 'Busan'}
{'_id': ObjectId('63b59a698291698b1cd1a781'), 'name': 'Jungkook John', 'phone_number': 8547962541, 'city': 'Busan'}
{'_id': ObjectId('63b59a698291698b1cd1a782'), 'name': 'Seokjin Kim',

In [18]:
old_data={'name':'Taehyung Kim'}                  ## updating the existing data using upate_many query
new_data={"$set":{'country':'Korea','job_role':'Kpop_Idol'}}
contact_details.update_many(old_data,new_data)

for document in contact_details.find():
    print(document)

{'_id': ObjectId('63b59a698291698b1cd1a77a'), 'name': 'Arjun Pandey', 'phone_number': 9845755542, 'city': 'Nanded'}
{'_id': ObjectId('63b59a698291698b1cd1a77b'), 'name': 'Dittya Thakre', 'phone_number': 7854857542, 'city': 'Pune'}
{'_id': ObjectId('63b59a698291698b1cd1a77c'), 'name': 'Ranjeet Chaudhry', 'phone_number': 8457961234, 'city': 'Mumbai'}
{'_id': ObjectId('63b59a698291698b1cd1a77d'), 'name': 'Suman Pandey', 'phone_number': 9845875462, 'city': 'Ahmednagar'}
{'_id': ObjectId('63b59a698291698b1cd1a77e'), 'name': 'Rishabh Pant', 'phone_number': 8754693214, 'city': 'Solapur'}
{'_id': ObjectId('63b59a698291698b1cd1a77f'), 'name': 'Taehyung Kim', 'phone_number': 5697541247, 'city': 'Daegu', 'country': 'Korea', 'job_role': 'Kpop_Idol'}
{'_id': ObjectId('63b59a698291698b1cd1a780'), 'name': 'Park Jimin', 'phone_number': 6589741237, 'city': 'Busan'}
{'_id': ObjectId('63b59a698291698b1cd1a781'), 'name': 'Jungkook John', 'phone_number': 8547962541, 'city': 'Busan'}
{'_id': ObjectId('63b59

In [19]:
myquery = { "name": "Arjun Pandey","phone_number":9845755542,"city":"Nanded"}
contact_details.delete_one(myquery)

In [20]:
for document in contact_details.find():
    print(document)

{'_id': ObjectId('63b59a698291698b1cd1a77b'), 'name': 'Dittya Thakre', 'phone_number': 7854857542, 'city': 'Pune'}
{'_id': ObjectId('63b59a698291698b1cd1a77c'), 'name': 'Ranjeet Chaudhry', 'phone_number': 8457961234, 'city': 'Mumbai'}
{'_id': ObjectId('63b59a698291698b1cd1a77d'), 'name': 'Suman Pandey', 'phone_number': 9845875462, 'city': 'Ahmednagar'}
{'_id': ObjectId('63b59a698291698b1cd1a77e'), 'name': 'Rishabh Pant', 'phone_number': 8754693214, 'city': 'Solapur'}
{'_id': ObjectId('63b59a698291698b1cd1a77f'), 'name': 'Taehyung Kim', 'phone_number': 5697541247, 'city': 'Daegu', 'country': 'Korea', 'job_role': 'Kpop_Idol'}
{'_id': ObjectId('63b59a698291698b1cd1a780'), 'name': 'Park Jimin', 'phone_number': 6589741237, 'city': 'Busan'}
{'_id': ObjectId('63b59a698291698b1cd1a781'), 'name': 'Jungkook John', 'phone_number': 8547962541, 'city': 'Busan'}
{'_id': ObjectId('63b59a698291698b1cd1a782'), 'name': 'Seokjin Kim', 'phone_number': 4568721397, 'city': 'Seoul'}
{'_id': ObjectId('63b59a6